<h2> Convert code from one language to another using ibm-granite-code hosted on Replicate</h2>

<h3>Notebook goals:</h3>

The learning goals of this notebook are:

1. Connect to ibm-granite/granite-8b-code-instruct-128k hosted on Replicate, and use it to Generate code for Acceptance Test Python Code
2. Connect to ibm-granite/granite-20b-code-instruct-8k hosted on Replicate, and use it to Generate code for Acceptance Test Python Code


<h3>Prerequisites: </h3>

1. Create an account on Replicate.
2. Copy the Replicate API Token to an environment file (.env file) and place it in the same directory as this notebook. Environment variable can be named as `REPLICATE_API_TOKEN`.
3. Install `langchain` and `ibm-granite-community/utils` python packages using below pip command.

In [ ]:
%pip install langchain
%pip install git+https://github.com/ibm-granite-community/utils

## Introduction


<h3>Code conversion from one language to another</h3>

1. Designed for user, who inputs or selects code in one language or whole projects to be translated, including test if they exist, and specifies the other language in which the output is needed(recipe only).
2. Generates new code in output language that is functionally the same as the input language code. Also generates corresponding resources, such as associated test scenarios.


## Using a local model

<h3>Get the variables from .env file</h3>

This guide will demonstrate a basic inference call using the replicate package.

To establish an authenticated session, the code below imports necessary packages, and loads the environment variable `REPLICATE_API_TOKEN` from the .env file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API token directly from environment variables
replicate_api_token = os.getenv('REPLICATE_API_TOKEN')

if replicate_api_token:
    print('API token loaded successfully:', replicate_api_token)
else:
    print('Failed to load API token. Please check your .env file.')


## Using a remotely-hosted model

The Granite Code models are available on [Replicate](https://replicate.com/).

At the moment, they are only available to members of the Granite Code team.
Request an invite to get access.

This guide will demonstrate a basic inference call using the `replicate` package as well
as via LangChain.
In both cases, you will provide a [Replicate API Token](https://replicate.com/account/api-tokens).


<h3>Deployment ID and Model ID Specifications</h3>

Mention the Deployment ID as granite and Specify the Model ID to be used for building the use-case.
Also, add additional parameters to tune in as per the apt expectation from the model

In [356]:
deployment_id = "ibm/granite-dev"

model_id = "ibm/granite-8b-code-instruct:50da94a0b1b5d28e3161d1312077d856eb673b87e633438362e4820fed563444"

parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 1000,
    "min_new_tokens": 1,
    "max_length": 20,
    "stop_sequences": [""],
    "repetition_penalty": 2
}

<h2>Prompt Definition: Python to Javascript Conversion</h2>

We further need to specify the prompt as an input to the model with apt configuration by including within it the specification along with the correct requirements needed. 

For an instance, in this use case, we implement the prompt to contain an instruction for converting code from one language to another, namely from Python to Javascript to cover the behavior of application of open api onto code conversion aspect.

In [357]:
prompt1 = """Using the directions below, Generate only the javascript code for the python code in input and nothing else. 


Input: def print_n_times(n):
    for i in range(n):
        print("Hello World!")
Output: function print_n_times = (n) => {"Hello World!".repeat(n)};

Input:
def Fibonacci(n):
    # Check if input is 0 then it will
    # print incorrect input
    if n < 0:
        print("Incorrect input")
    # Check if n is 0
    # then it will return 0
    elif n == 0:
        return 0
    # Check if n is 1,2
    # it will return 1
    elif n == 1 or n == 2:
        return 1
    else:
        return Fibonacci(n-1) + Fibonacci(n-2)
print(Fibonacci(9))

Output:
function Fibonacci = (n) => {
    if (n < 0) {
        console.log("Incorrect input");
    } else if (n == 0) {
        return 0;
    } else if (n == 1 || n == 2) {
        return 1;
    } else {
        return Fibonacci(n-1) + Fibonacci(n-2);
    }
};
console.log(Fibonacci(9));

Input: # Write a Javascript function to translate the python code provided below:
def add_two_numbers(a,b):
    num1 = a
    num2 = b
    sum = num1 + num2
    print("Sum of", num1, "and", num2 , "is", sum)

Output:"""

### Replicate package

Python package replicate needs to be installed for being imported in the next step.

Python package replicate needs to be installed for being imported in the next step.

In [ ]:
%pip install replicate

<h3>Import all the necessary libraries and packages</h3>

Replicate package needs to be imported in order to be used for refering to the deployment for which the prompt is being directed to achieving the test/s generated.

In [359]:
import replicate

<h3>Error Handling and Cleaning Output</h3>

Next, using replicate, prediction is created with the prompt designed as per the configuration in the previous step. Furthermore, the exception handling is curated onto the response of the prompt to generate clean output for the user in terms of the test generated inclusing error handling.

In [ ]:
deployment1 = replicate.deployments.get(deployment_id)
print("prompt", prompt1)
prediction1 = deployment1.predictions.create(
  input={"prompt": prompt1, "params" : parameters}
)

prediction1.wait()

if (prediction1.output[0].__contains__('mock')): 
  prompt_response1 = 'The provided input specification for code translation seems corrupted. Kindly check again and reattampt.'
else:
  prompt_response1 = (prediction1.output[0].split('<|endoftext|>')[0].split('\n\nAnswer:\n')[-1])
print(prompt_response1)

<h3>Prompt preparation for Test case scenario creations for converted code</h3>

Further, we will create a new prompt to generate a test case for the code translated using the following:

In [ ]:

if (prediction1 != 'The provided input specification for code translation seems corrupted. Kindly check again and reattampt.'):
    prompt2 = """Generate the test for the code provided in input. \n Input:""" + prompt_response1 + """\nOutput:"""

print(prompt2)

<h3>Fetching Acceptance Test as response to the prompt</h3>

Next, using replicate, prediction is created with the new prompt designed as per the configuration in the previous step. And, the exception handling is curated onto the response of the prompt to generate clean output for the user in terms of the test generated including error handling.

In [ ]:
if (prediction1 != 'The provided input specification for code translation seems corrupted. Kindly check again and reattampt.'):
  deployment2 = replicate.deployments.get(deployment_id)
  print("prompt", prompt2)
  prediction2 = deployment2.predictions.create(
    input={"prompt": prompt2, "params" : parameters}
  )

  prediction2.wait()
  if (prediction2.output[0].__contains__('mock')): 
    print('The test case for the function translated cannot generated. Please check the code and modify as needed or kindly check again and reattampt.')
  else:
    print((prediction2.output[0].split('<|endoftext|>')[0]))

<h2>Prompt Definition: Java to Python Conversion</h2>

We further need to specify the prompt as an input to the model with apt configuration by including within it the specification along with the correct requirements needed. 

For an instance, in this use case, we implement the prompt to contain an instruction for converting code from one language to another, namely from Java to Python to cover the behavior of application of open api onto code conversion aspect.

In [363]:
prompt3 = """Using the directions below, Generate only the python code for the java code in input and nothing else. 

Input: 
public class Array {
    public static void main(String[] args) {
        int[] array = {1, 2, 3, 4, 5};
        for (int element: array) {
            System.out.println(element);
        }
    }
}

Output:
numbers = [1, 2, 3, 4, 5]
for number in numbers:
	print(number)

Input:
import java.io.*;
class Fibonnaci {

    // Function to print the fibonacci series

    static int fib(int n)

    {

        // Base Case

        if (n <= 1)

            return n;

        // Recursive call

        return fib(n – 1) + fib(n – 2);

    }

    // Driver Code

    public static void main(String args[])

    {

        // Given Number N

        int N = 10;

        // Print the first N numbers

        for (int i = 0; i < N; i++) {

            System.out.print(fib(i) + ” “);

        }
    }
}    

Output:
def Fibonacci(n):
    # Check if input is 0 then it will
    # print incorrect input
    if n < 0:
        print("Incorrect input")
    # Check if n is 0
    # then it will return 0
    elif n == 0:
        return 0
    # Check if n is 1,2
    # it will return 1
    elif n == 1 or n == 2:
        return 1
    else:
        return Fibonacci(n-1) + Fibonacci(n-2)
print(Fibonacci(9))

Input: # Write a Python function to translate the java code provided below:
public class AsciiValue {
    public static void main(String[] args) {
        char ch = 'a';
        int ascii = ch;
        // You can also cast char to int
        int castAscii = (int) ch;
        System.out.println("The ASCII value of " + ch + " is: " + ascii);
        System.out.println("The ASCII value of " + ch + " is: " + castAscii);
    }
}

Output:"""

### Replicate package

Python package replicate needs to be installed for being imported in the next step.

In [ ]:
%pip install replicate

<h3>Import all the necessary libraries and packages</h3>

Replicate package needs to be imported in order to be used for refering to the deployment for which the prompt is being directed to achieving the test/s generated.

In [365]:
import replicate

<h3>Error Handling and Cleaning Output</h3>

Next, using replicate, prediction is created with the prompt designed as per the configuration in the previous step. Furthermore, the exception handling is curated onto the response of the prompt to generate clean output for the user in terms of the test generated inclusing error handling.

In [ ]:
deployment3 = replicate.deployments.get(deployment_id)
print("prompt", prompt3)
prediction3 = deployment3.predictions.create(
  input={"prompt": prompt3, "params" : parameters}
)

prediction3.wait()

if (prediction1.output[0].__contains__('mock')): 
  prompt_response3 = 'The provided input specification for code translation seems corrupted. Kindly check again and reattampt.'
else:
  prompt_response3 = (prediction3.output[0].split('<|endoftext|>')[0].split('\n\nAnswer:\n')[-1].split('# Test the function')[0])
print(prompt_response3)

<h3>Prompt preparation for Test case scenario creations for converted code</h3>

Further, we will create a new prompt to generate a test case for the code translated using the following:

In [ ]:

if (prediction3 != 'The provided input specification for code translation seems corrupted. Kindly check again and reattampt.'):
    prompt4 = """Generate only the test for the code provided in input. \n Input:""" + prompt_response3 + """\nOutput:"""

print(prompt4)

<h3>Fetching Acceptance Test as response to the prompt</h3>

Next, using replicate, prediction is created with the new prompt designed as per the configuration in the previous step. And, the exception handling is curated onto the response of the prompt to generate clean output for the user in terms of the test generated including error handling.

In [ ]:
if (prediction3 != 'The provided input specification for code translation seems corrupted. Kindly check again and reattampt.'):
  deployment4 = replicate.deployments.get(deployment_id)
  print("prompt", prompt4)
  prediction4 = deployment4.predictions.create(
    input={"prompt": prompt4, "params" : parameters}
  )

  prediction4.wait()
  if (prediction4.output[0].__contains__('mock')): 
    print('The test case for the function translated cannot generated. Please check the code and modify as needed or kindly check again and reattampt.')
  else:
    print((prediction4.output[0].split('<|endoftext|>')[0]))